In [85]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample
from cmath import nan
import json

In [86]:
#define the relevant path to the current batch you wish to parse
gap_name='pilot_two_week' #dont change this as this parsing sciprt only handles the same day batches.  

project_name='birds'
experiment_name='experiment 1'
batch_name='batch 1'

PATH_TO_BATCH=path.Path.cwd().parent.parent.parent / 'projects' / project_name / experiment_name / 'collected data' / gap_name / batch_name
PATH_TO_BATCH_DATA = PATH_TO_BATCH / 'data'

if PATH_TO_BATCH.exists():
    print('batch folder found at:\n' ,PATH_TO_BATCH)
else: 
    print('path to batch is non existent:\n',PATH_TO_BATCH)

batch folder found at:
 c:\Users\User\Desktop\visual_memory_project\visual-memory-project\projects\birds\experiment 1\collected data\pilot_two_week\batch 1


In [87]:


qualification_name_for_testin='UPDATE-completed encoding successfully ' #note that the -space- after the title is improtant as for some reason this is how the qualification name is defined

qualification_name_for_entire_experiment='UPDATE-completed birds memory rep'



In [88]:
#this section loads the shelf dict in the state it is after finishing the testing session for this batch. 
shelf_dict_after_test_name=PATH_TO_BATCH / 'shelf after encoding session closed.txt' #define the name of the relevant shelf for this stage
with open(shelf_dict_after_test_name) as f:
    data = f.read()
shelf_dict = json.loads(data)
shelf_dict

{'A2541C8MY0BYV3': [0, 1675267981506],
 'A3RCX3IQ8L6HHW': [1, 1675268029753],
 'ALCPF5NANBDSZ': [2, 1675268048545],
 'A2LI71A9ZRA199': [3, 1675268051443],
 'A1AMGHYG5PT0L2': [3, 1675268055067],
 'AX9LZ0N55O19E': [4, 1675268071765],
 'A28HB7240OFGEW': [5, 1675268100057],
 'A3S10MQO4RXHM8': [6, 1675268108691],
 'A1A71I3P1FXWCD': [999, 999, 999],
 'A1JJYY622DGE5L': [8, 1675268140997],
 'A10YV3PH9V9KQP': [9, 1675268157717],
 'A72YC75DZ8OLE': [999, 999, 999],
 'A146QN6EU8LAEI': [10, 1675268305246],
 'A34PI07YOEU3ZS': [999, 999, 999],
 'A29C1SVJECJ1V8': [999, 999, 999],
 'AOMYFDVVIRWN2': [999, 999, 999],
 'A1TIKB6CAQFH2B': [999, 999, 999],
 'A3QHZCELFJ3BSN': [11, 1675268724743],
 'A1HCJKXZTLF6VI': [7, 1675269155877],
 'A3OUAR84WQO03Y': [12, 1675269275539],
 'A3QRCBVXYI6AAK': [13, 1675269432919, 1675273510306],
 'ACKG8OU1KHKO2': [999, 999, 999],
 'AUZP4FPD6UTNU': [15, 1675269450862],
 'A2DX27FIUVZYB5': [14, 1675269782794],
 'A1SARRYNMZDVXB': [16, 1675269873675],
 'A3QPVH0DRW0ZQV': [17, 167527

In [89]:
#this section reads all the relevant files: 
if (PATH_TO_BATCH / 'Batch_encoding_batch_results.csv').exists():
    encoding_df=pd.read_csv(PATH_TO_BATCH / 'Batch_encoding_batch_results.csv')
else: 
    print(f'no "Batch_encoding_batch_results.csv" was found in data')

if (PATH_TO_BATCH / 'Batch_testing_batch_results.csv').exists():
    testing_df=pd.read_csv(PATH_TO_BATCH / 'Batch_testing_batch_results.csv')
else: 
    print(f'no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)')
if (PATH_TO_BATCH / 'Batch_workers.csv').exists():
    workers_df=pd.read_csv(PATH_TO_BATCH / 'Batch_workers.csv')
else:
    print('No AMAZON worker list was found: please make sure to upload one and name it correctly')



no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)


In [90]:
def find_largest_consequtive_repetition(responses_array):
#add a stop value so the loop will use the logic also if the end of the responses is all repeating: 
    responses_array=np.append(responses_array,[99])
    longest_rep_dict=dict()
    last_input=responses_array[0]
    consequtive_rep_counter=0
    for response in responses_array[1:]: 
        if response==last_input: #if consequtive rep: 
            consequtive_rep_counter=consequtive_rep_counter+1 #add to counter
        else: #once a new entry is in the vector - store the counter rep information 
            if last_input in longest_rep_dict.keys(): #if it was allready stored in our dictionary
                if longest_rep_dict[last_input]<consequtive_rep_counter: #replace the counter only if it is higher than what stored in dict
                    longest_rep_dict[last_input]=consequtive_rep_counter
            else: #this entry is not yet in the dictionary: add it 
                longest_rep_dict[last_input]=consequtive_rep_counter
            
            consequtive_rep_counter=0
            last_input=response
    return max(longest_rep_dict.values())

In [91]:
def process_worker_results(PATH_TO_BATCH_DATA,subject_name):
    cur_sub=pd.read_csv(PATH_TO_BATCH_DATA / subject_name)
    sub_demographics=cur_sub[['workID','Age','Gender']].iloc[0]


    #extract demo related information: 

    #if participant did not click on the arrow during the demo a 'demo_encoding_response.rt' colmumn wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('demo_encoding_response.rt' in cur_sub.columns):
        cur_sub['demo_encoding_response.rt']=np.nan
    demo_columns=['demo_encoding_loop.thisTrialN','DemoImage','DemoCorrect','demo_encoding_response.rt','demo_encoding_response.keys']

    sub_demo_information=cur_sub[demo_columns]
    empty_inds=sub_demo_information.loc[sub_demo_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_information=sub_demo_information.drop(index=empty_inds)


    #extract the demo test columns: 
    demo_test_columns=['demo_test_response.keys','demo_test_response.corr','demo_test_response.rt','demo_test_loop.thisTrialN','DemoImage1','DemoImage2','DemoCorrectTest']
    sub_demo_test_information=cur_sub[demo_test_columns]
    empty_inds=sub_demo_test_information.loc[sub_demo_test_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_test_information=sub_demo_test_information.drop(index=empty_inds)

    demo_df=pd.concat([sub_demo_information.reset_index(),sub_demo_test_information.reset_index()],axis=1)

    #extract real experiment related information: 

    #if participant did not click on the arrow during the encoding a 'test_encoding_response.rt' column wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('test_encoding_response.rt' in cur_sub.columns):
        cur_sub['test_encoding_response.rt']=np.nan


    encoding_related_columns=['test_encoding_response.keys','test_encoding_response.corr','trials.thisTrialN','target_image','pair','layer','correct','test_encoding_response.rt','key_resp_end.keys']
    sub_encoding_information=cur_sub[encoding_related_columns]
    #encoding section ends with a key press of the space key: so seperate this phase by finding this space key row
    end_of_section_ind=np.where(sub_encoding_information['key_resp_end.keys']=='space')[0][0]
    sub_encoding_information=sub_encoding_information.iloc[0:end_of_section_ind]
    #remove all the rows that precede the real encoding phase: 
    empty_inds=sub_encoding_information.loc[sub_encoding_information.isnull().apply(lambda x: all(x), axis=1)].index
    sub_encoding_information=sub_encoding_information.drop(index=empty_inds).reset_index()

    #extract real experiment TEST related information: 
    #test_related_columns=['layer','correct','test_test_response.keys','test_test_response.corr','test_test_response.rt','trials_2.thisRepN','trials_2.thisTrialN','trials_2.thisN','trials_2.thisIndex','trials_2.ran','image1','image2']
    #sub_test_information=cur_sub[test_related_columns].iloc[end_of_section_ind+2:-1]


    subject_dictionary=dict()
    subject_dictionary['demographics']=sub_demographics
    subject_dictionary['demo_df']=demo_df
    subject_dictionary['encoding_df']=sub_encoding_information
    #subject_dictionary['test_df']=sub_test_information

    return subject_dictionary



In [92]:

#get all results with Encoding information - extract the arrow attention check accuracy and RT (RT is currently not usd as a criterion)

#this section extract the list of participants from the downloaded results files (and not via the workers or session list csvs) 
# - it will create the qualification_df (a table with information on the worker ids and encoding behavior of all participants that we have files for)
all_filenames=[file for file in PATH_TO_BATCH_DATA.iterdir() if 'csv' in file.name and 'ENCODING' in file.name]

#print(f'current csv files:\n{all_filenames}')

qualification_for_test_df=pd.DataFrame(columns=['workerID','arrow_acc','mean_arrow_RT'])
for subject_csv in all_filenames:



    #thojs is a patch for the case of saving non complete files (some are ecompletly eomptyu, and otherd dont have some important demo columns)
    if (PATH_TO_BATCH_DATA / path.Path(subject_csv)).stat().st_size <= 5:
        print(f' subject: {subject_csv} did not have a demo column and thus disqualified')
        continue

    cur_sub=pd.read_csv(PATH_TO_BATCH_DATA / path.Path(subject_csv))
    #get rid of participants with encoding files that are basiscly empty:
    if (not('demo_test_response.keys' in cur_sub.columns)):
        print(f' subject: {subject_csv} did not have a demo column and thus disqualified')
        continue
        


    subject_dict=process_worker_results(PATH_TO_BATCH_DATA,path.Path(subject_csv))
    cur_sub_encoding=subject_dict['encoding_df']

    sname=subject_csv.name.split('_')[1] #change according to actuall format. 

    #get only attention check related info: 
    cur_sub_encoding=cur_sub_encoding[['test_encoding_response.keys','correct','test_encoding_response.rt']].copy().dropna(how = 'all')
    if len(cur_sub_encoding)==0: #empty - the participant didnt respond on the arrow at all
        arrow_acc=0
        RT=nan
    else: 
        RT=cur_sub_encoding['test_encoding_response.rt'].mean()
        arrow_acc=(cur_sub_encoding['correct']==cur_sub_encoding['test_encoding_response.keys']).mean()
    sub_df=pd.DataFrame({'workerID':sname,'arrow_acc':arrow_acc,'mean_arrow_RT':RT},index=[sname])
    qualification_for_test_df=pd.concat([qualification_for_test_df,sub_df],axis=0)

#the following part update the qualification_df with information on wether the participant id exists in the amazon workers list: 

#change participants qualifications if they exists in the workers list based on thier encoding arrow accuracy
qualification_for_test_df['in_encoding_workers_list']=nan

for curr_worker_ID in qualification_for_test_df.index:
    if curr_worker_ID in encoding_df['WorkerId'].values:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(1)
    else:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(0)


## the section below seperate between 4 types of participants: 
    * qualified and in the workers list
    * qualified and not in the workers list 
    * disqualified and in the worker list
    * disqualified and not in the worker list

In [93]:
qualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('qualified participants')
qualified_df

qualified participants


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A10JXOU89D5RXR,A10JXOU89D5RXR,1.0,0.492380,1.0
A10YV3PH9V9KQP,A10YV3PH9V9KQP,0.6,0.756367,1.0
A110KENBXU7SUJ,A110KENBXU7SUJ,1.0,0.637700,1.0
A11BQAB549UAEG,A11BQAB549UAEG,1.0,1.757700,1.0
A146QN6EU8LAEI,A146QN6EU8LAEI,1.0,1.306600,1.0
A1AA32T87SP2NV,A1AA32T87SP2NV,1.0,1.377200,1.0
A1AMGHYG5PT0L2,A1AMGHYG5PT0L2,0.8,0.681775,1.0
A1HCJKXZTLF6VI,A1HCJKXZTLF6VI,1.0,1.128480,1.0
A1JJYY622DGE5L,A1JJYY622DGE5L,0.8,0.533880,1.0
A1NVPXNM1PWTXH,A1NVPXNM1PWTXH,1.0,1.117760,1.0


In [94]:
qualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('qualified participants that are not in the workers list (and thus we cant add a qualification for them)')
qualified_df_not_in_workers_list

qualified participants that are not in the workers list (and thus we cant add a qualification for them)


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A361512FNHY889,A361512FNHY889,1.0,1.40174,0.0


In [95]:
disqualified_df_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('disqualified participants that are in the workers list:')
disqualified_df_in_workers_list

disqualified participants that are in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A178HXQENEAI65,A178HXQENEAI65,0.4,0.73130,1.0
A1SARRYNMZDVXB,A1SARRYNMZDVXB,0.4,1.31685,1.0
A1SL10V8CDNI9H,A1SL10V8CDNI9H,0.4,1.58770,1.0
A262YEPP7TVFWA,A262YEPP7TVFWA,0.0,0.40370,1.0
A2RC45D8LPJ39B,A2RC45D8LPJ39B,0.4,0.07874,1.0


In [96]:
disqualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('disqualified participants that are NOT in the workers list:')
disqualified_df_not_in_workers_list

disqualified participants that are NOT in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A377O7EE412E7T,A377O7EE412E7T,0.0,NaN,0.0


In [97]:
qualified_df['type']='qualified in workers list'
qualified_df_not_in_workers_list['type']='qualified not in workers list'
disqualified_df_in_workers_list['type']='disqualified in workers list'
disqualified_df_not_in_workers_list['type']='disqualified not in workers list'
info_from_all_downloaded_datafiles=pd.concat([qualified_df,qualified_df_not_in_workers_list,disqualified_df_in_workers_list,disqualified_df_not_in_workers_list],axis=0)
info_summary_df=info_from_all_downloaded_datafiles.groupby('type').aggregate({'arrow_acc':'count'})
info_summary_df['worker_ids']=nan
info_summary_df['worker_ids']=info_summary_df['worker_ids'].astype(object)
if 'qualified in workers list' in info_summary_df.index: 
    info_summary_df.at['qualified in workers list','worker_ids']=qualified_df.index.to_list()
if 'qualified not in workers list' in info_summary_df.index:    
    info_summary_df.at['qualified not in workers list','worker_ids']=qualified_df_not_in_workers_list.index.to_list()
if 'disqualified in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified in workers list','worker_ids']=disqualified_df_in_workers_list.index.to_list()
if 'disqualified not in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified not in workers list','worker_ids']=disqualified_df_not_in_workers_list.index.to_list()

info_summary_df

,arrow_acc,worker_ids
type,,
disqualified in workers list,5,"[A178HXQENEAI65, A1SARRYNMZDVXB, A1SL10V8CDNI9..."
disqualified not in workers list,1,[A377O7EE412E7T]
qualified in workers list,48,"[A10JXOU89D5RXR, A10YV3PH9V9KQP, A110KENBXU7SU..."
qualified not in workers list,1,[A361512FNHY889]


In [98]:
#update the workers list with the relevant qualification: 
new_shelf_dict=shelf_dict.copy()

#add the qualification for all eligible participants for the test session: 
ind_in_workers_df=[]
print('\nupdating qualifications for participants elligable for test session\n',list(qualified_df.index))
for worker_id in qualified_df.index:
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin].index[0])

print('\nupdating qualifications for participants elligable for test session that are not in the workers list\n',list(qualified_df_not_in_workers_list.index))
for worker_id in qualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#remove from experiment all those that disqualified (so they wont be able to take the test session): 
print('\nupdating qualifications for participants that were disqualified for test session and are in the workers list\n',list(disqualified_df_in_workers_list.index))
for worker_id in disqualified_df_in_workers_list.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment].index[0])
        
        if worker_id in shelf_dict.keys(): #change thier shelf entree (so they wont waste a valid csv number)
            prev_entriees=shelf_dict[worker_id]
            cur_entriees=prev_entriees.copy()
            cur_entriees[0]=999
            new_shelf_dict[worker_id]=cur_entriees
            print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#handle those that disqualified but are not in the amazon worker list: (change thier shelf entries to 999)
print('\nhandling participants that exists in the shelf but not in the amazon workers list (reseting thier shelf entriees)\n',list(disqualified_df_not_in_workers_list.index))
for worker_id in disqualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees.copy()
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#print the two relevant columns of the updated workers_list: 
print('these are all the participants that will be updated in the workers list:')
workers_df.loc[ind_in_workers_df,['Worker ID',qualification_name_for_testin,qualification_name_for_entire_experiment]]



updating qualifications for participants elligable for test session
 ['A10JXOU89D5RXR', 'A10YV3PH9V9KQP', 'A110KENBXU7SUJ', 'A11BQAB549UAEG', 'A146QN6EU8LAEI', 'A1AA32T87SP2NV', 'A1AMGHYG5PT0L2', 'A1HCJKXZTLF6VI', 'A1JJYY622DGE5L', 'A1NVPXNM1PWTXH', 'A206WLS4KMBH86', 'A21YNTPZSVT4QX', 'A23KAJRDVCVGOE', 'A2541C8MY0BYV3', 'A2APG8MSLJ6G2K', 'A2CZ5PS567SA10', 'A2DRJL37OEZ3B9', 'A2DX27FIUVZYB5', 'A2LI71A9ZRA199', 'A2N9U74YIPDQ9F', 'A2NCVGGFYM2KWD', 'A2OL5U1YWPFBDI', 'A2OT9ZJC4MQBS8', 'A2V6E7SGEU0EFD', 'A2VO8C41JJIQY9', 'A2VY5GKTXHLC8Q', 'A39WZD2C8BPP01', 'A3CCEIJXJ10GST', 'A3DW5DN6ZR69OQ', 'A3GH8ENBIFBGY', 'A3I40B0FATY8VH', 'A3OUAR84WQO03Y', 'A3PDGYYLZ5WQ96', 'A3QG7EQGHGDO5W', 'A3QHZCELFJ3BSN', 'A3QPVH0DRW0ZQV', 'A3RCX3IQ8L6HHW', 'A3RVHUY67SVXQV', 'A3S10MQO4RXHM8', 'A3TMVAECAVF9X5', 'A9LSEP71DNP4O', 'AB3920J1HAG7C', 'AE33JO53WTHZQ', 'ALCPF5NANBDSZ', 'AOMSOIGOYPFXX', 'AUZP4FPD6UTNU', 'AVSPYLB9KOHMX', 'AX9LZ0N55O19E']

updating qualifications for participants elligable for test session that 

,Worker ID,UPDATE-completed encoding successfully,UPDATE-completed birds memory rep
12,A10JXOU89D5RXR,1.0,NaN
19,A10YV3PH9V9KQP,1.0,NaN
22,A110KENBXU7SUJ,1.0,NaN
28,A11BQAB549UAEG,1.0,NaN
86,A146QN6EU8LAEI,1.0,NaN
191,A1AA32T87SP2NV,1.0,NaN
198,A1AMGHYG5PT0L2,1.0,NaN
327,A1HCJKXZTLF6VI,1.0,NaN
359,A1JJYY622DGE5L,1.0,NaN
435,A1NVPXNM1PWTXH,1.0,NaN


In [99]:
#save the worker list to upload (to update the qualifications) -> this is what you upload to amazon to update the qualifications of relevant participants
workers_df.to_csv(PATH_TO_BATCH/'Batch_workers_for_upload_after_encoding.csv',index=False)

In [100]:
#this section remove participants that were assigned with an unavailble csv from the shelf (also not changing qualifications for them at all so they could come back to another encoding session in the future) 
max_csv_number=62
participants_exceeding_available_csvs=[]
for key in shelf_dict.keys():
    cur_values=shelf_dict[key]
    if (cur_values[0]>max_csv_number) and (not(cur_values[0]>=998)):
        participants_exceeding_available_csvs.append(key)
        if key in new_shelf_dict.keys():
            del new_shelf_dict[key]

print(f'the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of {max_csv_number}:\nthis probably means that they did not even see the encoding of the experiment and should be considered as never participated in our\n{participants_exceeding_available_csvs}')

the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of 62:
this probably means that they did not even see the encoding of the experiment and should be considered as never participated in our
[]


In [101]:
#this section gives 999 to participants that are in the shelf but for some reason are not in the batch_results_file (so they probably saw the experiment, but we dont know how they performed during encoding so we discard them)
for worker_id in encoding_df['WorkerId']:
    if worker_id not in qualification_for_test_df.index:
        change_msg='- is not in the shelf'
        if worker_id in new_shelf_dict.keys():
            cur_values=new_shelf_dict[worker_id]
            new_values=cur_values.copy()
            new_values[0]=999
            new_shelf_dict[worker_id]=new_values
            change_msg=f'- shelf entry changed from: {cur_values} to {new_values}'

        print(f'{worker_id} did not have an encoding file ' + change_msg)

A72YC75DZ8OLE did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
AOMYFDVVIRWN2 did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
ASI7RTAD1SG4L did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A2GUSVGAKNEGQF did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A3RFPUOU1XWS09 did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A19JA6ETNMXVYY did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A1FKCQ6015OLCI did not have an encoding file - shelf entry changed from: [56, 1675357759148] to [999, 1675357759148]


In [102]:
#this section find participants that have a valid shelf entree but are not in the worker list (at all) 
#going over the worker list and not the results list because it is possible these participant were entered into the shelf due to a previous batchwoekre
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id not in workers_df['Worker ID'].values:
            new_values=cur_values.copy()
            new_values[0]=999
            change_msg=f' - shelf entry changed from: {cur_values} to {new_values}'
            new_shelf_dict[worker_id]=new_values
            print(f'{worker_id} absent from WORKERS LIST ' + change_msg)


A28HB7240OFGEW absent from WORKERS LIST  - shelf entry changed from: [5, 1675268100057] to [999, 1675268100057]
A3QRCBVXYI6AAK absent from WORKERS LIST  - shelf entry changed from: [13, 1675269432919, 1675273510306] to [999, 1675269432919, 1675273510306]
A34YNEN8UUI3CD absent from WORKERS LIST  - shelf entry changed from: [36, 1675282729685] to [999, 1675282729685]
A5P15BDXCVG32 absent from WORKERS LIST  - shelf entry changed from: [45, 1675296191339, 1675307260248] to [999, 1675296191339, 1675307260248]


In [103]:
#this section extracts the names of the remaining valid participants in the shelf: 
valid_participants_in_shelf_for_test=[]
valid_participants_csvnumber_in_shelf_for_test=[]
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id in workers_df['Worker ID'].values:
            valid_participants_in_shelf_for_test.append(worker_id)
            valid_participants_csvnumber_in_shelf_for_test.append(cur_values[0])

valid_shelf_participants_info_df=pd.DataFrame({'Worker ID':valid_participants_in_shelf_for_test,'numeric csv':valid_participants_csvnumber_in_shelf_for_test},index=valid_participants_in_shelf_for_test)
valid_shelf_participants_info_df

,Worker ID,numeric csv
A2541C8MY0BYV3,A2541C8MY0BYV3,0
A3RCX3IQ8L6HHW,A3RCX3IQ8L6HHW,1
ALCPF5NANBDSZ,ALCPF5NANBDSZ,2
A2LI71A9ZRA199,A2LI71A9ZRA199,3
A1AMGHYG5PT0L2,A1AMGHYG5PT0L2,3
AX9LZ0N55O19E,AX9LZ0N55O19E,4
A3S10MQO4RXHM8,A3S10MQO4RXHM8,6
A1JJYY622DGE5L,A1JJYY622DGE5L,8
A10YV3PH9V9KQP,A10YV3PH9V9KQP,9
A146QN6EU8LAEI,A146QN6EU8LAEI,10


In [104]:
valid_shelf_participants_info_df.join(qualification_for_test_df)

,Worker ID,numeric csv,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A2541C8MY0BYV3,A2541C8MY0BYV3,0,A2541C8MY0BYV3,1.0,0.579800,1.0
A3RCX3IQ8L6HHW,A3RCX3IQ8L6HHW,1,A3RCX3IQ8L6HHW,1.0,0.490680,1.0
ALCPF5NANBDSZ,ALCPF5NANBDSZ,2,ALCPF5NANBDSZ,1.0,0.499280,1.0
A2LI71A9ZRA199,A2LI71A9ZRA199,3,A2LI71A9ZRA199,1.0,1.405820,1.0
A1AMGHYG5PT0L2,A1AMGHYG5PT0L2,3,A1AMGHYG5PT0L2,0.8,0.681775,1.0
AX9LZ0N55O19E,AX9LZ0N55O19E,4,AX9LZ0N55O19E,1.0,1.144000,1.0
A3S10MQO4RXHM8,A3S10MQO4RXHM8,6,A3S10MQO4RXHM8,1.0,0.998620,1.0
A1JJYY622DGE5L,A1JJYY622DGE5L,8,A1JJYY622DGE5L,0.8,0.533880,1.0
A10YV3PH9V9KQP,A10YV3PH9V9KQP,9,A10YV3PH9V9KQP,0.6,0.756367,1.0
A146QN6EU8LAEI,A146QN6EU8LAEI,10,A146QN6EU8LAEI,1.0,1.306600,1.0


In [105]:
#this is the updated shelf: <<<<<< copy >>>>> from here and place in the shelf location in pavlovia.org
import json
json.dumps(new_shelf_dict)

'{"A2541C8MY0BYV3": [0, 1675267981506], "A3RCX3IQ8L6HHW": [1, 1675268029753], "ALCPF5NANBDSZ": [2, 1675268048545], "A2LI71A9ZRA199": [3, 1675268051443], "A1AMGHYG5PT0L2": [3, 1675268055067], "AX9LZ0N55O19E": [4, 1675268071765], "A28HB7240OFGEW": [999, 1675268100057], "A3S10MQO4RXHM8": [6, 1675268108691], "A1A71I3P1FXWCD": [999, 999, 999], "A1JJYY622DGE5L": [8, 1675268140997], "A10YV3PH9V9KQP": [9, 1675268157717], "A72YC75DZ8OLE": [999, 999, 999], "A146QN6EU8LAEI": [10, 1675268305246], "A34PI07YOEU3ZS": [999, 999, 999], "A29C1SVJECJ1V8": [999, 999, 999], "AOMYFDVVIRWN2": [999, 999, 999], "A1TIKB6CAQFH2B": [999, 999, 999], "A3QHZCELFJ3BSN": [11, 1675268724743], "A1HCJKXZTLF6VI": [7, 1675269155877], "A3OUAR84WQO03Y": [12, 1675269275539], "A3QRCBVXYI6AAK": [999, 1675269432919, 1675273510306], "ACKG8OU1KHKO2": [999, 999, 999], "AUZP4FPD6UTNU": [15, 1675269450862], "A2DX27FIUVZYB5": [14, 1675269782794], "A1SARRYNMZDVXB": [999, 1675269873675], "A3QPVH0DRW0ZQV": [17, 1675270229654], "A2GUSVGAK